In [ ]:
# Imports
from typing import Any, Optional, List
import Adafruit_PCA9685
import time
import math
from pathlib import Path
import sys

def find_project_root(start: Path = Path.cwd(), marker: str = "pyproject.toml") -> Optional[Path]:
    for p in [start] + list(start.parents):
        if (p / marker).is_file():
            return p
    return None

PROJECT_ROOT: Optional[Path] = find_project_root()
if PROJECT_ROOT:
    proj_path = str(PROJECT_ROOT)
    if proj_path not in sys.path:
        sys.path.insert(0, proj_path)
else:
    print(f"Warning: pyproject.toml not found in parents of {Path.cwd()}; sys.path unchanged")

In [ ]:
from src.servo.helpers_servo import angle_to_counts, counts_to_angle

In [5]:
# Convert angle in degrees to PCA9685 counts (0-4095)
def angle_to_counts(angle: float,
                    angle_min: int = ANGLE_MIN,
                    angle_max: int = ANGLE_MAX,
                    ms_min: float = SERVO_MIN_MS,
                    ms_max: float = SERVO_MAX_MS,
                    freq: int = FREQ) -> int:
    """Convert an angle in degrees to 12-bit PCA9685 counts.

    Angle is clamped to [angle_min, angle_max].
    Returns an integer count in [0, 4095].
    """
    angle_deg: float = max(angle_min, min(angle_max, float(angle)))
    pulse_ms: float = ms_min + (angle_deg - angle_min) / (angle_max - angle_min) * (ms_max - ms_min)
    counts: int = int(pulse_ms / (1000.0 / freq) * 4096)
    return counts

In [6]:
class ServoController:
    """Controller for a single servo connected to PCA9685.

    Naming conventions: type-first tokens for attributes (e.g., ``pos_channel``,
    ``pos_current_angle``) and full type hints on attributes and methods.
    """

    def __init__(self, drv_pwm: Any, pos_channel: int, pos_center: int = 90) -> None:
        self.drv_pwm: Any = drv_pwm
        self.pos_channel: int = pos_channel
        self.pos_center: int = pos_center
        self.pos_current_angle: int = pos_center  # assume starting at center
        # Move servo to the initial center position
        self.drv_pwm.set_pwm(self.pos_channel, 0, angle_to_counts(self.pos_current_angle))

    def move_to_center(self) -> None:
        """Move servo to its center position and update state."""
        self.drv_pwm.set_pwm(self.pos_channel, 0, angle_to_counts(self.pos_center))
        self.pos_current_angle = self.pos_center

    def move_to(self, target_angle: int, speed_deg_per_sec: float = 60.0, step_dt: float = 0.02) -> None:
        """Move servo to ``target_angle`` at ``speed_deg_per_sec`` using steps of ``step_dt`` seconds.

        This method updates ``pos_current_angle`` when the motion completes.
        """
        start_angle: int = self.pos_current_angle
        delta: float = float(target_angle - start_angle)
        duration: float = abs(delta) / float(speed_deg_per_sec) if speed_deg_per_sec > 0 else 0.0
        if duration <= 0.0:
            # Immediate move
            self.drv_pwm.set_pwm(self.pos_channel, 0, angle_to_counts(target_angle))
            self.pos_current_angle = target_angle
            return

        steps: int = max(1, int(duration / step_dt))
        dt: float = duration / steps
        for step_idx in range(1, steps + 1):
            angle_deg: float = start_angle + delta * (step_idx / steps)
            self.drv_pwm.set_pwm(self.pos_channel, 0, angle_to_counts(angle_deg))
            time.sleep(dt)
        self.pos_current_angle = target_angle

In [7]:
# Demo parameters (constants)
CHANNEL: int = 0
CENTER_DEG: int = 90
ANGLE_SHIFT_DEG: int = 60
SPEED_DEG_PER_SEC: float = 60.0  # degrees per second

In [8]:
# Create the servo controller for testing/demo
ctl_servo: ServoController = ServoController(drv_pwm, CHANNEL, pos_center=CENTER_DEG)

In [18]:
print(f"Moving CW {ANGLE_SHIFT_DEG}° at {SPEED_DEG_PER_SEC}°/s")
ctl_servo.move_to(CENTER_DEG + ANGLE_SHIFT_DEG, speed_deg_per_sec=SPEED_DEG_PER_SEC)

print("Waiting 1s...")
time.sleep(1)

Moving CW 60° at 60.0°/s


Waiting 1s...


In [17]:
print("Returning to center")
ctl_servo.move_to(CENTER_DEG, speed_deg_per_sec=SPEED_DEG_PER_SEC)

Returning to center
